In [ ]:
import json
import sklearn
import numpy
import random
import scipy
import sklearn.datasets
import sklearn.feature_extraction.text
import sklearn.linear_model
import sklearn.naive_bayes
import sklearn.metrics
import sklearn.utils
import string
import io
import re
import vocabulary

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import StanfordNERTagger
#import matplotlib.pyplot as plt

import os
java_path = "C:/Program Files/Java/jdk1.8.0_131/bin/java.exe"
os.environ['JAVAHOME'] = java_path

embedding_file, embedding_dimensions, embedding_cache = \
    "glove.6B.50d.txt", 50, "embedding.npz"

#st = StanfordNERTagger('C:\stanford-ner\classifiers\english.all.3class.distsim.crf.ser.gz', 'C:\stanford-ner\stanford-ner.jar', encoding='utf-8')


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
with open('articles_with_texts_final.json') as data_file:
    articles = json.load(data_file)
article_keys = articles[u'data'].keys()
db = {}

# with open('1054/mrc2.dct') as f:
#     for line in f:
#         #line = f.readline()
#         match_word = re.search(r'((\w)*)\|',line[51:], re.S)
#         #print line
#         match_img = int(line[31:34]) #:44
        
#         if match_img != 0:
#             db[match_word.group(1)] = match_img

In [ ]:
vocab_file, vocab_file_type = "vocab.pkl", "pickle"

def iterable() :
    for key in article_keys:
        tokens = nltk.word_tokenize(articles[u'data'][u'text'])
        for o in tokens:
            yield o

made_vocabulary = False
if made_vocabulary :
    v = vocabulary.Vocabulary.load(vocab_file, file_type=vocab_file_type)
else: 
    v = vocabulary.Vocabulary.from_iterable(iterable(),file_type=vocab_file_type)
    v.save(vocab_file)
v.stop_growth()

In [ ]:
embedding_file, embedding_dimensions, embedding_cache = "glove.6B.50d.txt", 50, "embedding.npz"

def load_sparse_csr(filename):
    loader = numpy.load(filename)
    return scipy.sparse.csr_matrix((loader['data'], loader['indices'], loader['indptr']), shape = loader['shape'])

def save_sparse_csr(filename, array):
    numpy.savez(filename, data = array.data, indices=array.indices, indptr =array.indptr, shape=array.shape )
    
def build_sparse_embedding(vocab, glovefile, d) :
    remaining_vocab = vocab.keyset()
    embeddings = np.zeros((len(remaining_vocab), d))
    
    with open(glovefile) as glovedata :
        fileiter = glovedata.readlines()
        rows = []
        columns = []
        values = []
        
        for line in fileiter :
            line = line.replace("\n","").split(" ")
            try:
                glove_key, nums = line[0], [float(x.strip()) for x in line[1:]]
                for word in (glove_key, glove_key.capitalize(), glove_key.upper()) :
                    if word in remaining_vocab :
                        columns.append(np.arange(len(nums)))
                        rows.append(np.full(len(nums), vocab[word]))
                        values.append(np.array(nums))
                        remaining_vocab.remove(word)
            except Exception as e:
                print("{} broke. exception: {}. line: {}.".format(word, e, x))

        print("{} words were not in glove".format(len(remaining_vocab)))
        return scipy.sparse.coo_matrix((np.concatenate(values),
                                        (np.concatenate(rows),
                                         np.concatenate(columns))),
                                        shape=(len(vocab), d)).tocsr()

made_embedding = False
if made_embedding :
    e = load_sparse_csr(embedding_cache)
else: 
    e = build_sparse_embedding(v, embedding_file, embedding_dimensions)
    newsreader.save_sparse_csr(embedding_cache, e)

In [ ]:
vectorizer = \
sklearn.feature_extraction.text.TfidfVectorizer(sublinear_tf=True, 
                                                max_df=0.5, 
                                                stop_words='english')
_ = vectorizer.fit(articles[u'data'][k][u'text'] for k in article_keys[:4000])

In [ ]:
T_keys = [k for k in article_keys if (articles[u'data'][k]['most_emailed'] or articles[u'data'][k]['most_shared'] or \
                articles[u'data'][k]['most_viewed'])]
F_keys = [k for k in article_keys if not (articles[u'data'][k]['most_emailed'] or articles[u'data'][k]['most_shared'] or \
                articles[u'data'][k]['most_viewed'])]

In [ ]:
#def feature1(c_articles) : #c_articles is [articles[u'data'][k] for k in keys]

#author_vectorizer = sklearn.feature_extraction.text.CountVectorizer()
#_ = author_vectorizer.fit(string.replace(string.replace(ca[u'byline'][u'original'], ' ', ''), '.', '') if ca[u'byline'] != [] else "" for ca in [articles[u'data'][k] for k in article_keys])

def author(cas):
    return author_vectorizer.transform(string.replace(string.replace(ca[u'byline'][u'original'], ' ', ''), '.', '') if ca[u'byline'] != [] else "" for ca in cas)
    
# truth with noise
def truth(cas):
    if len(cas) == 4000 :
        return numpy.reshape([(1 if random.randint(1,4) > 1 else 0) if k in T_keys else (1 if random.randint(1,4) == 1 else 0) for k in article_keys[:4000]], (-1, 1))
    else :
        return numpy.reshape([(1 if random.randint(1,4) > 1 else 0) if k in T_keys else (1 if random.randint(1,4) == 1 else 0) for k in article_keys[4000:]], (-1, 1))

def img_rating(cas):
    text = [ca[u'text'] for ca in cas]
    imagery_val = [sum(db[word] if word in db else 0 for word in article)/(700.0*len(article)+1.0) for article in text]
    return (numpy.matrix(imagery_val)).reshape(-1,1)

NE_count_file = 'NE_counts.txt'
    
def NE_count(text):
    tokenized_text = word_tokenize(text)
    t = st.tag(tokenized_text)
    n_persons = 0.0
    n_org = 0.0
    n_loc = 0.0
    if len(t) == 0 :
        return [0.0, 0.0, 0.0]
    (a, b) = t[0]
    if b == u'PERSON' :
        n_persons += 1
    elif b == u'LOCATION' :
        n_loc += 1
    elif b == u'ORGANIZATION' :
        n_org += 1
    prev = b
    for (a, b) in t[1:] :
        if b != prev :
            prev = b
            if b == u'PERSON' :
                n_persons += 1
            elif b == u'LOCATION' :
                n_loc += 1
            elif b == u'ORGANIZATION' :
                n_org += 1
    return [n_persons/len(tokenized_text), n_loc/len(tokenized_text), n_org/len(tokenized_text)]

def named_entity(keys):
    NE_count_dict = {}
    if NE_count_file != None :
        with open(NE_count_file, 'r') as f:
            for line in f:
                l = line.split()
                NE_count_dict[l[0].encode()] = [float(l[1]), float(l[2]), float(l[3])]
        return numpy.matrix([NE_count_dict[key] for key in keys])
    else:
        return numpy.matrix([NE_count(articles[u'data'][key][u'text']) for key in keys])

def sparse(cas):
    text = [ca[u'text'] for ca in cas]
    x = numpy.zeros((len(text), embedding_dimensions))
    for i in range(len(text)) :
        vec = numpy.zeros(embedding_dimensions)
        for token in nltk.word_tokenize(articles[u'data'][u'text']):
            if token in v.keyset():
                vec = numpy.add(e[v[token]], vec)
        x[i] = vec
    
features = [sparse, author, img_rating]
features_k = [named_entity]


def getX(key_list):
    #x = vectorizer.transform(articles[u'data'][k][u'text'] for k in key_list)
    c_articles = [articles[u'data'][k] for k in key_list]
    x = sparse(c_articles)
    for f in features :
        x_new = f(c_articles)
        x = scipy.sparse.hstack([x, x_new])
    for f in features_k :
        x_new = f(key_list)
        x = scipy.sparse.hstack([x, x_new])
    return x

In [ ]:
#[string.replace(string.replace(ca[u'byline'][u'original'], ' ', ''), '.', '') if ca[u'byline'] != [] else "" for ca in [articles[u'data'][k] for k in article_keys[:4000]]]

In [ ]:
X_train = getX(article_keys[:4000])
y_train = [k in T_keys for k in article_keys[:4000]]
X_dev = getX(article_keys[4000:])
y_dev = [k in T_keys for k in article_keys[4000:]]

In [ ]:
classifier = \
sklearn.linear_model.SGDClassifier(loss="log",
                                   penalty="elasticnet",
                                   n_iter=5)

_ = classifier.fit(X_train, y_train)

In [ ]:
pred = classifier.predict(X_dev)
print sklearn.metrics.accuracy_score(y_dev, pred)
print sklearn.metrics.confusion_matrix(y_dev, pred)

In [ ]:
index_to_word = vectorizer.get_feature_names()
best_elts = numpy.argpartition(classifier.coef_, classifier.coef_.size - 20)[0][-20:]
best_words = [index_to_word[x] if x < len(index_to_word) else x-len(index_to_word) for x in best_elts]
best_words

In [ ]:
worst_elts = numpy.argpartition(classifier.coef_, 19)[0][:20]
worst_words = [index_to_word[x] if x < len(index_to_word) else x-len(index_to_word) for x in worst_elts]
worst_words

In [ ]:
#with open('NE_counts.txt', 'w') as f:
#    for i in range(4000):
#        f.write(article_keys[i] + ' ' + str(NE_train[i][0]) + ' ' + str(NE_train[i][1]) + ' ' + str(NE_train[i][2]) + '\n')
#    for i in range(len(NE_dev)):
#        f.write(article_keys[4000+i] + ' ' + str(NE_dev[i][0]) + ' ' + str(NE_dev[i][1]) + ' ' + str(NE_dev[i][2]) + '\n')

In [ ]:
getX(article_keys[:4000])